In [1]:
from pyvi import ViTokenizer #For split vietnamese words
import pandas as pd #For reading xlsx file
from gensim.parsing.preprocessing import strip_non_alphanum, strip_multiple_whitespaces,preprocess_string, split_alphanum, strip_short, strip_numeric
import re
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score
import string

In [2]:
def raw_text_preprocess(raw):
    raw = re.sub(r"http\S+", "URL_Change", raw)
    raw = re.sub(r"https\S+", "URL_Change", raw)
    raw = re.sub(r"[0-2][0-9]/[0-1][0-9]/[0-9][0-9][0-9][0-9]", "DATE_Change", raw)
    raw = re.sub(r"[0-2][0-9]-[0-1][0-9]-[0-9][0-9][0-9][0-9]", "DATE_Change", raw)
    
    raw = re.sub(r"[0-9]\S+", "NUMBER_Change", raw)
    raw = ViTokenizer.tokenize(raw)
    raw = strip_short(raw, minsize=2)
    return raw

In [3]:
document = []
label = []
def read_file(file):
    with open(file, 'r', encoding = 'utf-8') as f:

        for line in f:
            document.append(line[7: ])
            if ('non' in line[: 6]):
                label.append(0)
            else:
                label.append(1)

read_file('data_mail.txt')        
read_file('data_email.txt')
print(len(document))


421


In [4]:
# def change(raw):
#     raw_cop = raw
#     token = raw.split(" ")
#     links = ('http:', '.com', '.org', '.vn', '@', 'https:')
#     for i, char in enumerate(token):
# #         print(char)
#         for link in links:
#             if (link in char):
#                 token[i] = 'URL'
#                 break
#     for i, tok in enumerate(token):
#         if (tok[0].isdigit()):
#             token[i] = 'number_change'
#     return token

In [5]:
document = [raw_text_preprocess(d) for d in document]
document_test = document[: 90]
label_test = label[: 90]
document = document[90 :]
label = label[90 :]
print(document_test)

['Thân gửi Anh Chị FPT trân_trọng giới_thiệu đến Anh Chị sự_kiện NGÀY CÔNG_NGHỆ FPT NUMBER_Change FPT TECHDAY NUMBER_Change Đến với sự_kiện lần này khách tham_dự sẽ có cơ_hội trải nghiệm những công_nghệ hàng_đầu của FPT và những câu_chuyện thực_tế về chuyển_đổi số của các doanh_nghiệp Lần đầu_tiên tại Việt_Nam chiếc xe tự_hành Made in Viet_Nam trở_thành Người vận_chuyển khách mời tham_gia sự_kiện Lần đầu_tiên Trợ_lý ảo tổng_đài được phát_triển từ nền_tảng trí_tuệ nhân_tạo Made in Viet_Nam đảm_nhiệm vị_trí ghi_danh và xác_nhận hàng ngàn khách mời Tương_lai thế_giới số mở ra từ những câu_chuyện thực_tế về cuộc hành_trình chuyển_đổi số của những người Việt_Nam tiên_phong Đặc_biệt sinh_viên và những người đam_mê công_nghệ có_thể đăng_ký tham_gia đấu_trường công_nghệ kịch_tính bước lên đài vinh_quang và giành tổng giải_thưởng lên đến NUMBER_Change_triệu đồng_tiền_mặt Để nhận được vé tham_dự Anh Chị vui_lòng ĐĂNG_KÝ MIỄN_PHÍ ngay tại ĐÂY Thời_gian Thứ_Năm ngày DATE_Change Địa_điểm Trung_tâm 

In [6]:
def dict_word(document):
    res = {}
    words = {}
    conten = 0
    for doc in document:
#             token = change(doc)
            token = doc.split(" ")
            for tok in token:
                if tok in res.keys():
                    res[tok] = res[tok] + 1
                else:
                    res.update({tok : 1})
            for key, value in res.items():
                if (value >= 3) and (value <= (5 * len(document))):
                    words.update({key : value})
            word = {}
            for tok in token:
                if tok in word.keys():
                    word[tok] = word[tok] + 1
                else:
                    word.update({tok : 1})
            conten = conten + len(word)
    words = sorted(words.keys())
    print(len(words))
    return (words, conten)

In [7]:
def read_data(document, label, words, conten):
    word_list = []
#     conten = 0
    for doc in document:
#         token = change(doc)
        token = doc.split(" ")
        word = {}
        for tok in token:
            if tok in word.keys():
                word[tok] = word[tok] + 1
            else:
                word.update({tok : 1})
        word_list.append(word)
    j = 0
    dat = np.zeros((conten, 3), dtype = int)
    for i, word in enumerate(word_list):
        for key, value in word.items():
            if(key in words):
                dat[j, :] = np.array([int(i), int(words.index(key)), int(value)])
                j = j + 1
    data = coo_matrix((dat[:, 2], (dat[:, 0] , dat[:, 1] )),shape=(len(label), len(words)))
    return (data, label)

In [8]:
(words, conten) = dict_word(document)
(train_data, train_label) = read_data(document, label, words, conten)
(test_data, test_label) = read_data(document_test, label_test, words, conten)
clf = MultinomialNB()
clf.fit(train_data, train_label)
y_pred = clf.predict(test_data)
print('Training size =',train_data.shape[0],'. accuracy =', accuracy_score(y_pred,test_label)*100)

2364
Training size = 331 . accuracy = 87.77777777777777
